# CGT FINAL EXPERIMENT LAUNCHER

## AUDIT COMPLIANCE
- ✅ Notebook = imports only
- ✅ All logic in modules
- ✅ Single evaluation pipeline
- ✅ No new methodological decisions

## MODELS
1. K-Lighting Numerical Parity (reference)
2. K-Lighting AGI v2
3. CGT Paper Ready
4. Hybrid / Hybrid Active

---

In [1]:
# @title 1. Setup Environment
!pip install -q sentence-transformers datasets scipy POT

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.8 MB/s eta 0:00:00
PyTorch: 2.9.0+cu126
CUDA: True
GPU: NVIDIA A100-SXM4-40GB


In [5]:
# @title 2. Upload and Extract cgt_project.zip
from google.colab import files
import zipfile
import os

# Clean previous extraction
!rm -rf /content/cgt_project
!rm -rf /content/checkpoints
print("Cleaned previous extraction.")

# Upload
print("\nUpload cgt_project_FINAL.zip:")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\nExtracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as z:
            z.extractall('/content')
        print(f"Extracted: {filename}")
        # Remove zip after extraction
        os.remove(filename)

# Verify structure
print("\n" + "="*60)
print("VERIFICATION")
print("="*60)
print("\n1. Root contents:")
!ls -la /content/cgt_project/
print("\n2. src/ contents:")
!ls -la /content/cgt_project/src/
print("\n3. src/cgt/ exists?")
!ls /content/cgt_project/src/cgt/ 2>/dev/null && echo "✅ src/cgt/ found!" || echo "❌ ERROR: src/cgt/ NOT FOUND!"
print("\n4. Key file check:")
!ls /content/cgt_project/src/cgt/models/cgt_hardened.py 2>/dev/null && echo "✅ cgt_hardened.py found!" || echo "❌ ERROR: cgt_hardened.py NOT FOUND!"

Cleaned previous extraction.

Upload cgt_project_FINAL.zip:


Saving cgt_project_FINAL.zip to cgt_project_FINAL.zip

Extracting cgt_project_FINAL.zip...
Extracted: cgt_project_FINAL.zip

VERIFICATION

1. Root contents:
total 68
drwxr-xr-x 9 root root 4096 Jan 19 16:07 .
drwxr-xr-x 1 root root 4096 Jan 19 16:07 ..
drwxr-xr-x 2 root root 4096 Jan 19 16:07 configs
drwxr-xr-x 2 root root 4096 Jan 19 16:07 docs
drwxr-xr-x 9 root root 4096 Jan 19 16:07 experiments
-rw-r--r-- 1 root root 6731 Jan 19 16:07 LICENSE
drwxr-xr-x 2 root root 4096 Jan 19 16:07 notebooks
-rw-r--r-- 1 root root 2864 Jan 19 16:07 pyproject.toml
-rw-r--r-- 1 root root 8266 Jan 19 16:07 README.md
-rw-r--r-- 1 root root  509 Jan 19 16:07 requirements.txt
drwxr-xr-x 2 root root 4096 Jan 19 16:07 scripts
-rw-r--r-- 1 root root 2173 Jan 19 16:07 setup.py
drwxr-xr-x 3 root root 4096 Jan 19 16:07 src
drwxr-xr-x 2 root root 4096 Jan 19 16:07 tests

2. src/ contents:
total 16
drwxr-xr-x  3 root root 4096 Jan 19 16:07 .
drwxr-xr-x  9 root root 4096 Jan 19 16:07 ..
drwxr-xr-x 11 root root 40

In [6]:
# @title 3. Add Project to Path
import sys
import os

# Check if structure is correct
src_path = '/content/cgt_project/src'
exp_path = '/content/cgt_project/experiments'

if not os.path.exists(f'{src_path}/cgt'):
    print("ERROR: src/cgt/ not found!")
    print("Checking alternative paths...")
    # Maybe ZIP was extracted differently
    if os.path.exists('/content/src/cgt'):
        src_path = '/content/src'
        exp_path = '/content/experiments'
        print(f"Found at: {src_path}")
    else:
        raise FileNotFoundError("Cannot find cgt module. Check ZIP extraction.")

sys.path.insert(0, src_path)
sys.path.insert(0, exp_path)
print(f"Added to path: {src_path}")
print(f"Added to path: {exp_path}")

# Verify imports
from cgt.models.cgt_hardened import CGTStudentHardened
from cgt.geometry.lorentz_hardened import LorentzSubstrateHardened
print("✅ Core modules imported")

from unified import run_all_replications, train_hybrid, load_stsb_data, load_hybrid_data
from unified.hybrid_active_trainer import train_hybrid_active
print("✅ Unified modules imported")
print("✅ Hybrid Active trainer imported")

Added to path: /content/cgt_project/src
Added to path: /content/cgt_project/experiments
✅ Core modules imported
✅ Unified modules imported
✅ Hybrid Active trainer imported


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:42: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return self.getter()


In [7]:
# @title 4. Configuration
from pathlib import Path

# Output directory
OUTPUT_BASE = Path("/content/experiment_outputs")
OUTPUT_BASE.mkdir(exist_ok=True)

# Experiment mode for hybrid cell
# EXPERIMENT_MODE semantics:
# "hybrid"        -> legacy hybrid (pre-active loss)
# "hybrid_active" -> hybrid ACTIVE loss
#                    (7-term loss: CGT + K-Lighting + Ψ-SLM + TDA)
EXPERIMENT_MODE = "hybrid"  # @param ["hybrid", "hybrid_active"]
SKIP_PSI_SLM = True  # @param {type:"boolean"}

# Hybrid Active Loss Configuration (only used when EXPERIMENT_MODE = "hybrid_active")
LOSS_CONFIG = dict(
    lambda_contrastive=1.0,
    lambda_distill=0.5,
    lambda_topological=0.1,
    lambda_forman=0.1,
    lambda_coherence=0.1,
    lambda_lipschitz=0.05,
    lambda_gw=0.2,
)

print(f"Experiment Mode: {EXPERIMENT_MODE}")
print(f"Skip PSI_SLM: {SKIP_PSI_SLM}")
print(f"Output: {OUTPUT_BASE}")
if EXPERIMENT_MODE == "hybrid_active":
    print(f"Loss Config: {LOSS_CONFIG}")

Experiment Mode: hybrid
Skip PSI_SLM: True
Output: /content/experiment_outputs


In [8]:
# @title 5. Run Replications (3 models)
from unified import run_all_replications, load_stsb_data

# Load data
print("Loading STS-B data...")
data = load_stsb_data()

# Run replications
print("\nRunning replications...")
replication_results = run_all_replications(
    output_base=OUTPUT_BASE / "outputs",
    data=data,
    skip_psi_slm=SKIP_PSI_SLM,
)

print("\n✅ Replications complete")

Loading STS-B data...
[INFO] Loading STS-B dataset...


README.md: 0.00B [00:00, ?B/s]

train.jsonl.gz:   0%|          | 0.00/278k [00:00<?, ?B/s]

validation.jsonl.gz:   0%|          | 0.00/86.4k [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/63.2k [00:00<?, ?B/s]

...


############################################################
# REPLICATION: k_light_agi_v2
############################################################


Epoch   1/20 | Loss: 0.0645 | Val ρ: 0.7787 | Best: 0.7787 (ep 1)
INFO:replication_k_light_agi_v2:Epoch   1/20 | Loss: 0.0645 | Val ρ: 0.7787 | Best: 0.7787 (ep 1)
Epoch   2/20 | Loss: 0.0350 | Val ρ: 0.7809 | Best: 0.7809 (ep 2)
INFO:replication_k_light_agi_v2:Epoch   2/20 | Loss: 0.0350 | Val ρ: 0.7809 | Best: 0.7809 (ep 2)
Epoch   3/20 | Loss: 0.0286 | Val ρ: 0.7863 | Best: 0.7863 (ep 3)
INFO:replication_k_light_agi_v2:Epoch   3/20 | Loss: 0.0286 | Val ρ: 0.7863 | Best: 0.7863 (ep 3)
Epoch   4/20 | Loss: 0.0270 | Val ρ: 0.7798 | Best: 0.7863 (ep 3)
INFO:replication_k_light_agi_v2:Epoch   4/20 | Loss: 0.0270 | Val ρ: 0.7798 | Best: 0.7863 (ep 3)
Epoch   5/20 | Loss: 0.0240 | Val ρ: 0.7910 | Best: 0.7910 (ep 5)
INFO:replication_k_light_agi_v2:Epoch   5/20 | Loss: 0.0240 | Val ρ: 0.7910 | Best: 0.7910 (ep 5)
Epoch   6/20 | Loss: 0.0221 | Val ρ: 0.7838 | Best: 0.7910 (ep 5)
INFO:replication_k_light_agi_v2:Epoch   6/20 | Loss: 0.0221 | Val ρ: 0.7838 | Best: 0.7910 (ep 5)
Epoch   7/20 | L


############################################################
# REPLICATION: cgt_paper_ready
############################################################


Epoch   1/25 | Loss: 0.0706 | Val ρ: 0.7727 | Best: 0.7727 (ep 1)
INFO:replication_cgt_paper_ready:Epoch   1/25 | Loss: 0.0706 | Val ρ: 0.7727 | Best: 0.7727 (ep 1)
Epoch   2/25 | Loss: 0.0412 | Val ρ: 0.7889 | Best: 0.7889 (ep 2)
INFO:replication_cgt_paper_ready:Epoch   2/25 | Loss: 0.0412 | Val ρ: 0.7889 | Best: 0.7889 (ep 2)
Epoch   3/25 | Loss: 0.0327 | Val ρ: 0.7908 | Best: 0.7908 (ep 3)
INFO:replication_cgt_paper_ready:Epoch   3/25 | Loss: 0.0327 | Val ρ: 0.7908 | Best: 0.7908 (ep 3)
Epoch   4/25 | Loss: 0.0303 | Val ρ: 0.8004 | Best: 0.8004 (ep 4)
INFO:replication_cgt_paper_ready:Epoch   4/25 | Loss: 0.0303 | Val ρ: 0.8004 | Best: 0.8004 (ep 4)
Epoch   5/25 | Loss: 0.0274 | Val ρ: 0.7946 | Best: 0.8004 (ep 4)
INFO:replication_cgt_paper_ready:Epoch   5/25 | Loss: 0.0274 | Val ρ: 0.7946 | Best: 0.8004 (ep 4)
Epoch   6/25 | Loss: 0.0254 | Val ρ: 0.7994 | Best: 0.8004 (ep 4)
INFO:replication_cgt_paper_ready:Epoch   6/25 | Loss: 0.0254 | Val ρ: 0.7994 | Best: 0.8004 (ep 4)
Epoch   7/


✅ Replications complete


In [9]:
# @title 6. Train Hybrid Model (dispatcher)
from unified import train_hybrid, load_hybrid_data
from unified.hybrid_active_trainer import train_hybrid_active

# Load data with mpnet teacher (768d)
print("Loading hybrid data (all-mpnet-base-v2)...")
hybrid_data = load_hybrid_data()

# Dispatcher based on EXPERIMENT_MODE
assert EXPERIMENT_MODE in {"hybrid", "hybrid_active"}, (
    f"Invalid EXPERIMENT_MODE: {EXPERIMENT_MODE}"
)

if EXPERIMENT_MODE == "hybrid":
    # Original hybrid trainer
    print("\nTraining hybrid model (original)...")
    hybrid_results = train_hybrid(
        output_dir=OUTPUT_BASE / "outputs" / "hybrid",
        data=hybrid_data,
    )
elif EXPERIMENT_MODE == "hybrid_active":
    # Hybrid Active trainer with 7-term loss
    print("\nTraining hybrid_active model (7-term loss)...")
    hybrid_results = train_hybrid_active(
        output_dir=OUTPUT_BASE / "outputs" / "hybrid_active",
        data=hybrid_data,
        **LOSS_CONFIG,
    )
else:
    raise ValueError(f"Unknown EXPERIMENT_MODE: {EXPERIMENT_MODE}")

print("\n✅ Hybrid training complete")

Loading hybrid data (all-mpnet-base-v2)...
[INFO] Loading STS-B dataset...
[INFO] Loading teacher: all-mpnet-base-v2 (768d) [PSI_SLM]...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

[INFO] Encoding train split...
[INFO] Encoding validation split...
[INFO] Encoding test split...


Seed: 42 (fixed)
INFO:hybrid_trainer:Seed: 42 (fixed)
INFO:hybrid_trainer:======================================================================
HYBRID MODEL TRAINING
INFO:hybrid_trainer:HYBRID MODEL TRAINING
INFO:hybrid_trainer:======================================================================

╔══════════════════════════════════════════════════════════════════════════════╗
║                         HYBRID MODEL DEFINITION                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ARCHITECTURE BASE: K-Lighting Numerical Parity                              ║
║  ├── Student: CGTStudentHardened (32d output)                                ║
║  ├── Substrate: LorentzSubstrateHardened (c=-1.0)                            ║
║  └── Hidden: 256d MLP                                                        ║
║                                                 

[INFO] Teacher (all-mpnet-base-v2) baseline Spearman: 0.8342

Training hybrid model (original)...


Epoch   1/25 | Loss: 236.7983 | Val ρ: 0.8069 | Best: 0.8069 (ep 1)
INFO:hybrid_trainer:Epoch   1/25 | Loss: 236.7983 | Val ρ: 0.8069 | Best: 0.8069 (ep 1)
Epoch   2/25 | Loss: 231.0119 | Val ρ: 0.8108 | Best: 0.8108 (ep 2)
INFO:hybrid_trainer:Epoch   2/25 | Loss: 231.0119 | Val ρ: 0.8108 | Best: 0.8108 (ep 2)
Epoch   3/25 | Loss: 253.6035 | Val ρ: 0.8086 | Best: 0.8108 (ep 2)
INFO:hybrid_trainer:Epoch   3/25 | Loss: 253.6035 | Val ρ: 0.8086 | Best: 0.8108 (ep 2)
Epoch   4/25 | Loss: 230.7354 | Val ρ: 0.8090 | Best: 0.8108 (ep 2)
INFO:hybrid_trainer:Epoch   4/25 | Loss: 230.7354 | Val ρ: 0.8090 | Best: 0.8108 (ep 2)
Epoch   5/25 | Loss: 230.7130 | Val ρ: 0.8118 | Best: 0.8118 (ep 5)
INFO:hybrid_trainer:Epoch   5/25 | Loss: 230.7130 | Val ρ: 0.8118 | Best: 0.8118 (ep 5)
Epoch   6/25 | Loss: 230.6977 | Val ρ: 0.8118 | Best: 0.8118 (ep 5)
INFO:hybrid_trainer:Epoch   6/25 | Loss: 230.6977 | Val ρ: 0.8118 | Best: 0.8118 (ep 5)
Epoch   7/25 | Loss: 230.6885 | Val ρ: 0.8101 | Best: 0.8118 (ep


✅ Hybrid training complete


In [10]:
# @title 7. Run Unified Evaluation
from unified.final_executor import run_final_execution

print("Running final evaluation...")
final_results = run_final_execution(
    output_base=OUTPUT_BASE,
    skip_psi_slm=SKIP_PSI_SLM,
)

print("\n✅ Evaluation complete")

Running final evaluation...
FINAL EXECUTION PIPELINE
Device: cuda
Output: /content/experiment_outputs

[PHASE 1] Loading data (MiniLM, 384d)...
[INFO] Loading teacher: all-MiniLM-L6-v2
[INFO] Loading STS-B dataset...
[INFO] Encoding train...
[INFO] Encoding validation...
[INFO] Encoding test...
[INFO] Teacher baseline: ρ = 0.8203

[PHASE 2] Loading data (mpnet, 768d)...
[INFO] Loading teacher: all-mpnet-base-v2
[INFO] Loading STS-B dataset...
[INFO] Encoding train...
[INFO] Encoding validation...
[INFO] Encoding test...
[INFO] Teacher baseline: ρ = 0.8342

[PHASE 3] Executing models...

######################################################################
# MODEL: k_light_numerical_parity
######################################################################

EVALUATING: k_light_numerical_parity

[1/4] Computing STS-B metrics...
  Test Spearman: 0.7637
  Test Pearson: 0.7711
  Val Spearman: 0.7928
  Retention: 93.1%

[2/4] Running falsification tests...
  F1 (Projection): FAIL (error=

In [11]:
# @title 8. Display Results Table

# Load results
results_path = OUTPUT_BASE / "tables" / "final_results.txt"
if results_path.exists():
    with open(results_path) as f:
        print(f.read())
else:
    print("Results not found. Run evaluation first.")

FINAL RESULTS TABLE
Generated: 2026-01-19T16:10:23.398465

--------------------------------------------------------------------------------------------------------------------------------------------------------
Modelo                         | Teacher                   | Dim Orig | Dim Comp | ρ (Spearman) |  Retention |      Storage |   Falsif | Obs            
--------------------------------------------------------------------------------------------------------------------------------------------------------
hybrid                         | all-mpnet-base-v2         |      768 |       33 |       0.7668 |      91.9% |     6408.8KB |      ✗✓✗ | Rank #1 ★      
k_light_numerical_parity       | all-MiniLM-L6-v2          |      384 |       33 |       0.7637 |      93.1% |     4099.3KB |      ✗✓✗ | Rank #2 ★      
k_light_agi_v2                 | all-MiniLM-L6-v2          |      384 |       33 |       0.7616 |      92.8% |     4098.9KB |      ✗✓✗ | Rank #3 ★      
cgt_paper_ready        

In [12]:
# @title 9. Create Final Delivery ZIP
import shutil

# Create delivery directory
DELIVERY_DIR = Path("/content/FINAL_DELIVERY")
DELIVERY_DIR.mkdir(exist_ok=True)

# Copy outputs
shutil.copytree(OUTPUT_BASE / "outputs", DELIVERY_DIR / "outputs", dirs_exist_ok=True)
shutil.copytree(OUTPUT_BASE / "tables", DELIVERY_DIR / "tables", dirs_exist_ok=True)
if (OUTPUT_BASE / "checkpoints").exists():
    shutil.copytree(OUTPUT_BASE / "checkpoints", DELIVERY_DIR / "checkpoints", dirs_exist_ok=True)

# Copy cgt_project
shutil.make_archive(
    str(DELIVERY_DIR / "cgt_project"),
    'zip',
    '/content/cgt_project'
)

# Create README
readme = f"""
# CGT FINAL DELIVERY

## Experiment Mode: {EXPERIMENT_MODE}

## Contents

- `cgt_project.zip` - Complete project code
- `outputs/` - Model checkpoints and training logs
- `tables/` - Results tables and evaluation data
- `checkpoints/` - Phase completion checkpoints

## Models Evaluated

1. K-Lighting Numerical Parity (reference)
2. K-Lighting AGI v2
3. CGT Paper Ready
4. {'Hybrid Active (7-term loss)' if EXPERIMENT_MODE == 'hybrid_active' else 'Hybrid'}

## Audit Compliance

- ✅ No code invented
- ✅ No metrics altered
- ✅ No inference without evidence
- ✅ Pipeline is unique and replicable
- ✅ State persisted in files
"""

with open(DELIVERY_DIR / "README.md", "w") as f:
    f.write(readme)

# Create final ZIP
shutil.make_archive(
    '/content/FINAL_DELIVERY',
    'zip',
    DELIVERY_DIR
)

print("✅ FINAL_DELIVERY.zip created")
!ls -lh /content/FINAL_DELIVERY.zip

✅ FINAL_DELIVERY.zip created
-rw-r--r-- 1 root root 19M Jan 19 16:10 /content/FINAL_DELIVERY.zip


In [13]:
# @title 10. Download Final Delivery
from google.colab import files

files.download('/content/FINAL_DELIVERY.zip')
print("\n✅ Download started")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download started


---

## FINAL DECLARATION

```
Declaro que:
- Nenhum código foi inventado
- Nenhuma métrica foi alterada
- Nenhuma inferência foi feita sem evidência explícita
- O pipeline de avaliação é único e replicável
- O estado do trabalho está persistido em arquivos
```

---